<a href="https://colab.research.google.com/github/NN198/LLM-Magic/blob/main/Generating_Detecting_Texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.4 MB/s eta 0:00:00


In [ ]:
#Importing used packages and libraries
from transformers import AutoConfig, AutoTokenizer,AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import numpy as np
import random
import openai #to use OpenAI api
import torch.nn.functional as F
from sklearn.metrics import auc,roc_curve

In [ ]:
# Another series of work attempted to build connections
# between ICL and gradient descent. Taking
# linear regression as a starting point, Akyürek
# et al. (2022) found that Transformer-based incontext
# learners can implement standard finetuning
# algorithms implicitly, and von Oswald et al.
# (2022) showed that linear attention-only Transformers
# with hand-constructed parameters and models
# learned by gradient descent are highly related.

# Loading and Instatiating the model for detection purposes

Vicuna-7B Model detector
*has been adversarially trained and known to produce best detection between human and AI text*

In [ ]:
# Assuming detector is your model and tokenizer is already defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cuda:0"
detector_modelcard = "TrustSafeAI/RADAR-Vicuna-7B"
#models - [Writer/camel-5b-hf,databricks/dolly-v2-3b]
detector = AutoModelForSequenceClassification.from_pretrained(detector_modelcard)

*Pretrained model classes and configuration*

In [ ]:
print(detector)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

**Model Configuration**

In [ ]:
detector_config = AutoConfig.from_pretrained(detector_modelcard)
print(detector_config)

RobertaConfig {
  "_name_or_path": "TrustSafeAI/RADAR-Vicuna-7B",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



*Tokenizing the model weights for performing paraphrasing and processing the ai texts before calculating the predictions*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(detector_modelcard)
detector.eval()
# detector.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

Completing the given prompts using different pretrained gpt models to test the robustness of the detector

In [ ]:
#Instruction and text inputs (Paper Reference -[Learning toWatermark LLM-generated Text via Reinforcement Learning] "Examples of C4 on Llama2-7b dataset", Examples of C4 on OPT-1.3b dataset)
instruction = "You will assist in completing the given text examples:"
raw_texts = ["The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald...",
             "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications...",
             "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take...",
             "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent..."]
human_texts = ["The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald Trump and North Korean leader Kim Jong Un will meet within weeks, White House spokesman Raj Shah said on Monday, as the administration seeks a deal with Pyongyang to rid North Korea of its nuclear weapons programme. The meeting would be the first of its kind between leaders of the countries, and potentially set in motion a long-awaited ”peace dialogue” that could lead to the first meeting of Moon Jae-in, the newly elected leader of South Korea, with the reclusive North. Advertisement For the latest headlines, follow our Google News channel online or via the app. The visit of Kim to South Korea for a summit on April",
               "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications. The first commercial system was the Solar Total Energy Project (STEP) in Shenandoah, Georgia, USA where a field of 114 parabolic dishes provided 50% of the process heating, air conditioning and electrical requirements for a clothing factory. This grid-connected cogeneration system provided 400 kW of electricity plus thermal energy in the form of 401 kW steam and 468 kW chilled water, and had a one-hour peak load thermal storage. Evaporation ponds are shallow pools that concentrate dissolved solids through evaporation. The use of evaporation ponds to obtain salt from sea water is one of the oldest applications of solar energy. Modern uses include concentrating brine solutions used in leach mining and removing dissolved solids from waste",
               "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take a bite out of the cost of living in the South Bay. The bill, AB 191, would allow cities to create a Community Facilities District, which would allow them to issue bonds to pay for infrastructure improvements. The bill would also allow cities to create a Community Facilities District to pay for infrastructure improvements. “The South Bay is a great place to live, work and raise a family, but the cost of living is too high,” Muratsuchi said in a statement. “AB 191 will help cities in the South Bay and across the state",
               "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent. Among the stated reasons were that Saddam's regime had tried to acquire nuclear material and had not properly accounted for biological and chemical material it was known to have previously possessed, and believed to still maintain. Both the possession of these weapons of mass destruction (WMD), and the failure to account for them, would violate the U.N. sanctions. The assertion about WMD was hotly advanced by the Bush administration from the beginning, but other major powers including China, France, Germany, and Russia remained unconvinced that Iraq was a threat and refused to allow passage of a UN Security Council resolution to authorize the use of force. Iraq permitted UN weapon inspectors in November 2002, who were continuing their work to assess the WMD claim when the Bush administration decided to proceed with war without UN authorization and told the inspectors to leave the"]

In [ ]:
class NLGClass:
  def __init__(self, api_key, raw_texts, instruction, model):
    openai.api_key = api_key
    self.human_texts = raw_texts
    self.instruction = instruction
    self.model = model


  def generate_text_completion(self, max_tokens, seed, temperature,top_p, n, frequency_penalty):
    ai_texts = []
    for text in self.human_texts:
      response = openai.ChatCompletion.create(
          model=self.model,
          messages=[
              {"role": "system", "content": self.instruction},
              {"role": "user", "content": text}
          ],
          max_tokens=max_tokens,
          seed=seed,
          temperature=temperature,
          top_p=top_p,
          n=n, #iterations
          stop=None,
          frequency_penalty=frequency_penalty
      )
      ai_texts.append(response.choices[0].message['content'].strip())
    return ai_texts

Generating completed texts from GPT-3.5 turbo

In [ ]:
api_key = "your-openai-key"
model = "gpt-3.5-turbo"
nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=0, temperature=0.5,top_p=0.1, n=1, frequency_penalty=0)
print(AI_texts)

# Paraphrasing human responses

In [ ]:
class ParaphraseNLG:
  @staticmethod
  def openai_response(text, openai_model):
    instruction = {"role": "system", "content": "Enhance the word choices in the sentence to sound more like that of a human."}
    prompt={"role": "user", "content": text}
    messages = [instruction, prompt]
    k_wargs = { "messages": messages, "model": openai_model}
    r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
    return r

In [ ]:
#Paraphrasing human text to worsen the quality of the text that can be compared with the AI generated texts
class ParaphraseNLGHumanText:
  @staticmethod
  def openai_response(text, openai_model):
    instruction = {"role": "system", "content": "Worsen the word choices in the sentence to sound less like that of a human"}
    prompt={"role": "user", "content": text}
    messages = [instruction, prompt]
    k_wargs = { "messages": messages, "model": openai_model}
    r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
    return r

In [ ]:
#These are the paraphrased responses generated from GPT 3.5
#Using gpt-4o to check the semantical differences from human text
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]
Paraphrased_responses_ht = [ParaphraseNLGHumanText.openai_response(i, "gpt-4o") for i in human_texts]

Calculating the log-probabilities for each instance in the batch

In [ ]:
#without gradient
# input will contain the raw output scores(logits in this case) of the probabilities generated by the detector
class textPrediction:
  @staticmethod
  def textInputFun(examples):
    with torch.no_grad():
      inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
      input = {k: v.to(device) for k, v in inputs.items()}
      output_prob = F.log_softmax(detector(**inputs).logits, -1)[:, 0].exp().tolist()
    print("No of input instances fed to the detector", len(examples))
    print(f"Probability of text examples: {output_prob}")
    return output_prob

In [ ]:
#Using gradients
class textPredictionwGrad:
    @staticmethod
    def textInputFun(examples):
        inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        # Ensure float tensors have requires_grad set to True
        for key, tensor in inputs.items():
            if tensor.dtype in [torch.float, torch.float32, torch.float64]:
                tensor.requires_grad_(True)

        # Ensure all inputs are on the correct device
        for key, tensor in inputs.items():
            inputs[key] = tensor.to(device)

        outputs = detector(**inputs)
        logits = outputs.logits
        output_prob = F.log_softmax(logits, -1)[:, 0].exp().tolist()
        print("No of input instances fed to the detector", len(examples))
        print(f"Probability of text examples: {output_prob}")
        return output_prob

In [ ]:
#Without gradients
print("Human text predictions from the detector")
human_preds = textPrediction.textInputFun(human_texts)
print("AI text predictions from the detector")
ai_preds = textPrediction.textInputFun(AI_texts)
print("Paraphrased text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses)
print("Paraphrased human text predictions")
paraphrased_preds_ht = textPrediction.textInputFun(Paraphrased_responses_ht)

Human text predictions from the detector
No of input instances fed to the detector 4
Probability of text examples: [0.22256025671958923, 0.1006336361169815, 0.9964590668678284, 0.0139209795743227]
AI text predictions from the detector
No of input instances fed to the detector 4
Probability of text examples: [0.9994514584541321, 0.9986514449119568, 0.9993454813957214, 0.9992238283157349]
Paraphrased text predictions
No of input instances fed to the detector 4
Probability of text examples: [0.9993172883987427, 0.997180700302124, 0.9993402361869812, 0.9991400241851807]
Paraphrased human text predictions
No of input instances fed to the detector 4
Probability of text examples: [0.3398597240447998, 0.034741152077913284, 0.9979898929595947, 0.012674978002905846]


Using gradients to determine the log probabilities

In [ ]:
#Using gradients

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector_modelcard = "TrustSafeAI/RADAR-Vicuna-7B"
detector = AutoModelForSequenceClassification.from_pretrained(detector_modelcard)
tokenizer = AutoTokenizer.from_pretrained(detector_modelcard)
detector.eval().to(device)

# Instruction and text inputs
instruction = "You will assist in completing the given text examples:"
raw_texts = [
    "The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald...",
    "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications...",
    "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take...",
    "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent..."
]
human_texts = [
    "The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald Trump and North Korean leader Kim Jong Un will meet within weeks, White House spokesman Raj Shah said on Monday, as the administration seeks a deal with Pyongyang to rid North Korea of its nuclear weapons programme. The meeting would be the first of its kind between leaders of the countries, and potentially set in motion a long-awaited ”peace dialogue” that could lead to the first meeting of Moon Jae-in, the newly elected leader of South Korea, with the reclusive North. Advertisement For the latest headlines, follow our Google News channel online or via the app. The visit of Kim to South Korea for a summit on April",
    "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications. The first commercial system was the Solar Total Energy Project (STEP) in Shenandoah, Georgia, USA where a field of 114 parabolic dishes provided 50% of the process heating, air conditioning and electrical requirements for a clothing factory. This grid-connected cogeneration system provided 400 kW of electricity plus thermal energy in the form of 401 kW steam and 468 kW chilled water, and had a one-hour peak load thermal storage. Evaporation ponds are shallow pools that concentrate dissolved solids through evaporation. The use of evaporation ponds to obtain salt from sea water is one of the oldest applications of solar energy. Modern uses include concentrating brine solutions used in leach mining and removing dissolved solids from waste",
    "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take a bite out of the cost of living in the South Bay. The bill, AB 191, would allow cities to create a Community Facilities District, which would allow them to issue bonds to pay for infrastructure improvements. The bill would also allow cities to create a Community Facilities District to pay for infrastructure improvements. “The South Bay is a great place to live, work and raise a family, but the cost of living is too high,” Muratsuchi said in a statement. “AB 191 will help cities in the South Bay and across the state",
    "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent. Among the stated reasons were that Saddam's regime had tried to acquire nuclear material and had not properly accounted for biological and chemical material it was known to have previously possessed, and believed to still maintain. Both the possession of these weapons of mass destruction (WMD), and the failure to account for them, would violate the U.N. sanctions. The assertion about WMD was hotly advanced by the Bush administration from the beginning, but other major powers including China, France, Germany, and Russia remained unconvinced that Iraq was a threat and refused to allow passage of a UN Security Council resolution to authorize the use of force. Iraq permitted UN weapon inspectors in November 2002, who were continuing their work to assess the WMD claim when the Bush administration decided to proceed with war without UN authorization and told the inspectors to leave the"
]

class NLGClass:
    def __init__(self, api_key, raw_texts, instruction, model):
        openai.api_key = api_key
        self.human_texts = raw_texts
        self.instruction = instruction
        self.model = model

    def generate_text_completion(self, max_tokens, seed, temperature, top_p, n, frequency_penalty):
        ai_texts = []
        for text in self.human_texts:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.instruction},
                    {"role": "user", "content": text}
                ],
                max_tokens=max_tokens,
                seed=seed,
                temperature=temperature,
                top_p=top_p,
                n=n,  # iterations
                stop=None,
                frequency_penalty=frequency_penalty
            )
            ai_texts.append(response.choices[0].message['content'].strip())
        return ai_texts

api_key = "your-openai-key"
model = "gpt-3.5-turbo"
nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=0, temperature=0.5, top_p=0.1, n=1, frequency_penalty=0)
print(AI_texts)

class ParaphraseNLG:
  @staticmethod
  def openai_response(text, openai_model):
    instruction = {"role": "system", "content": "Enhance the word choices in the sentence to sound more like that of a human."}
    prompt={"role": "user", "content": text}
    messages = [instruction, prompt]
    k_wargs = { "messages": messages, "model": openai_model}
    r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
    return r

# Paraphrasing human text to worsen the quality of the text that can be compared with the AI generated texts
class ParaphraseNLGHumanText:
    @staticmethod
    def openai_response(text, openai_model):
        instruction = {"role": "system", "content": "Worsen the word choices in the sentence to sound less like that of a human"}
        prompt = {"role": "user", "content": text}
        messages = [instruction, prompt]
        k_wargs = {"messages": messages, "model": openai_model}
        r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
        return r

# Using gradients
class textPredictionwGrad:
    @staticmethod
    def textInputFun(examples):
        inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Ensure float tensors have requires_grad set to True
        for key, tensor in inputs.items():
            if tensor.dtype in [torch.float, torch.float32, torch.float64]:
                tensor.requires_grad_(True)

        # Ensure all inputs are on the correct device
        for key, tensor in inputs.items():
            inputs[key] = tensor.to(device)

        outputs = detector(**inputs)
        logits = outputs.logits
        output_prob = F.log_softmax(logits, -1)[:, 0].exp().tolist()
        print("No of input instances fed to the detector", len(examples))
        print(f"Probability of text examples: {output_prob}")
        return output_prob

# Using gradients
print("Human text predictions from the detector using gradients")
human_preds_wg = textPredictionwGrad.textInputFun(human_texts)

print("AI text predictions from the detector using gradients")
ai_preds_wg = textPredictionwGrad.textInputFun(AI_texts)

# Assuming Paraphrased_responses and Paraphrased_responses_ht are defined somewhere in your code
# Example placeholders
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]
Paraphrased_responses_ht = [ParaphraseNLGHumanText.openai_response(i, "gpt-4o") for i in human_texts]

print("Paraphrased text predictions using gradients")
paraphrased_preds_wg = textPredictionwGrad.textInputFun(Paraphrased_responses)

print("Paraphrased human text predictions using gradients")
paraphrased_preds_ht_wg = textPredictionwGrad.textInputFun(Paraphrased_responses_ht)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


['Trump has expressed his willingness to meet with North Korean leader Kim Jong Un in the near future.', 'such as steam generation, water heating, and thermal energy storage. These technologies use mirrors or lenses to concentrate sunlight onto a receiver, where the energy is converted into heat. This heat can then be used in various industrial processes such as food processing, chemical production, and desalination. By harnessing the power of the sun, these solar concentrating technologies offer a sustainable and environmentally friendly alternative to traditional fossil fuel-based heating systems.', '...action against offshore oil drilling in California. The bill aims to protect the coastal communities and marine environment from the potential risks associated with oil drilling activities.', 'The Bush administration then turned its attention to Iraq, arguing that the need to remove Saddam Hussein from power had become urgent due to concerns about weapons of mass destruction and links

In [ ]:
def get_roc_metrics(human_preds, ai_pred):
      # human_preds is the ai-generated probabilities of human-text
      # ai_preds is the ai-generated probabiities of ai-text
      fpr, tpr, _ = roc_curve([0] * len(human_preds_wg) + [1] * len(ai_preds_wg), human_preds_wg + ai_preds_wg,pos_label=1)
      roc_auc = auc(fpr, tpr)
      return fpr.tolist(), tpr.tolist(), float(roc_auc)

In [ ]:
print("W/O Paraphrase Detection AUROC: ", get_roc_metrics(human_preds,ai_preds))
print("W/ Paraphrase Detection AUROC: ", get_roc_metrics(human_preds,paraphrased_preds))
print("W/ Paraphrase Detection AUROC: ", get_roc_metrics(human_preds,paraphrased_preds_ht))

W/O Paraphrase Detection AUROC:  ([0.0, 0.0, 0.0, 1.0], [0.0, 0.25, 1.0, 1.0], 1.0)
W/ Paraphrase Detection AUROC:  ([0.0, 0.0, 0.0, 0.25, 0.25, 1.0], [0.0, 0.25, 0.75, 0.75, 1.0, 1.0], 0.9375)
W/ Paraphrase Detection AUROC:  ([0.0, 0.0, 0.25, 0.25, 1.0, 1.0], [0.0, 0.25, 0.25, 0.5, 0.5, 1.0], 0.4375)


In [ ]:
print("W/O Paraphrase Detection AUROC with gradients: ", get_roc_metrics(human_preds_wg,ai_preds_wg))
print("W/ Paraphrase Detection AUROC with gradients: ", get_roc_metrics(human_preds_wg,paraphrased_preds_wg))
print("W/ Paraphrase Detection AUROC with gradients: ", get_roc_metrics(human_preds_wg,paraphrased_preds_ht))

W/O Paraphrase Detection AUROC with gradients:  ([0.0, 0.0, 0.0, 1.0], [0.0, 0.25, 1.0, 1.0], 1.0)
W/ Paraphrase Detection AUROC with gradients:  ([0.0, 0.0, 0.0, 1.0], [0.0, 0.25, 1.0, 1.0], 1.0)
W/ Paraphrase Detection AUROC with gradients:  ([0.0, 0.0, 0.0, 1.0], [0.0, 0.25, 1.0, 1.0], 1.0)


***Altering the text features and observing predictions***

Test 1- max_tokens=300, seed=1, temperature=0.8,top_p=0.1, n=1, frequency_penalty=-2

In [ ]:
model = "gpt-3.5-turbo"
nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=1, temperature=0.8,top_p=0.1, n=1, frequency_penalty=-2)
print(AI_texts)

['Trump has expressed his willingness to meet with North Korean leader Kim Jong Un to discuss denuclearization and peace on the Korean Peninsula.', 'such as steam generation, water heating, and thermal energy storage. These technologies use mirrors or lenses to concentrate sunlight onto a receiver, where the energy is converted into heat. The high temperatures achieved can be used in a variety of industrial processes, such as in the manufacturing, food, and chemical industries, as well as in power generation, desalination, and, in some cases,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', '...take action on the environmental impacts of the oil industry in the Hermosa Beach area.', 'The Bush administration then turned its attention to Iraq, arguing that the need to remove Saddam Hussein from power had become urgent.']


In [ ]:
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]
Paraphrased_responses_ht = [ParaphraseNLGHumanText.openai_response(i, "gpt-4o") for i in human_texts]

In [ ]:
print("Human text predictions from the detector")
human_preds = textPrediction.textInputFun(human_texts)
print("AI text predictions from the detector")
ai_preds = textPrediction.textInputFun(AI_texts)
print("Paraphrased text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses)
print("Paraphrased human text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses_ht)

Human text predictions from the detector
No of input instances fed to the detector 4
Probability of text examples: [0.22256025671958923, 0.1006336361169815, 0.9964590668678284, 0.0139209795743227]
AI text predictions from the detector
No of input instances fed to the detector 4
Probability of text examples: [0.9993743300437927, 0.9994033575057983, 0.9994151592254639, 0.9992238283157349]
Paraphrased text predictions
No of input instances fed to the detector 4
Probability of text examples: [0.99943608045578, 0.9828252196311951, 0.9995008707046509, 0.9991456270217896]
Paraphrased human text predictions
No of input instances fed to the detector 4
Probability of text examples: [0.016393421217799187, 0.054396893829107285, 0.9943987727165222, 0.010598534718155861]


Test 2- max_tokens=300, seed=2, temperature=1,top_p=0.1, n=1, frequency_penalty=-1

In [ ]:
model = "gpt-3.5-turbo"
nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=1, temperature=0.8,top_p=0.1, n=1, frequency_penalty=-1)
# print(AI_texts)
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]
print("Human text predictions from the detector")
human_preds = textPrediction.textInputFun(human_texts)
print("AI text predictions from the detector")
ai_preds = textPrediction.textInputFun(AI_texts)
print("Paraphrased text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses)
print("Paraphrased human text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses_ht)

Human text predictions from the detector
No of input instances fed to the detector 4
Probability of text examples: [0.22256025671958923, 0.1006336361169815, 0.9964590668678284, 0.0139209795743227]
AI text predictions from the detector
No of input instances fed to the detector 4
Probability of text examples: [0.9994514584541321, 0.9983538389205933, 0.9994996786117554, 0.9992238283157349]
Paraphrased text predictions
No of input instances fed to the detector 4
Probability of text examples: [0.9995185136795044, 0.9972115159034729, 0.9994076490402222, 0.9992938041687012]
Paraphrased human text predictions
No of input instances fed to the detector 4
Probability of text examples: [0.016393421217799187, 0.054396893829107285, 0.9943987727165222, 0.010598534718155861]


# With Camel-5b prerained model

In [ ]:
# Assuming detector is your model and tokenizer is already defined
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cuda:0"
from transformers import GPT2Tokenizer, GPT2Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
detector_modelcard = "Writer/camel-5b-hf"
#models - [Writer/camel-5b-hf,databricks/dolly-v2-3b]
detector = GPT2Model.from_pretrained(detector_modelcard)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
detector_config = AutoConfig.from_pretrained(detector_modelcard)
print(detector_config)

GPT2Config {
  "_name_or_path": "Writer/camel-5b-hf",
  "activation_function": "gelu",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.01,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 4096,
  "n_head": 32,
  "n_inner": 16384,
  "n_layer": 24,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 50258
}



In [ ]:

tokenizer = GPT2Tokenizer.from_pretrained(detector_modelcard)
detector.eval()
# detector.to(device)

GPT2Model(
  (wte): Embedding(50258, 4096)
  (wpe): Embedding(2048, 4096)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-23): 24 x GPT2Block(
      (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): GELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
class NLGClass:
  def __init__(self, api_key, raw_texts, instruction, model):
    openai.api_key = api_key
    self.human_texts = raw_texts
    self.instruction = instruction
    self.model = model


  def generate_text_completion(self, max_tokens, seed, temperature,top_p, n, frequency_penalty):
    ai_texts = []
    for text in self.human_texts:
      response = openai.ChatCompletion.create(
          model=self.model,
          messages=[
              {"role": "system", "content": self.instruction},
              {"role": "user", "content": text}
          ],
          max_tokens=max_tokens,
          seed=seed,
          temperature=temperature,
          top_p=top_p,
          n=n, #iterations
          stop=None,
          frequency_penalty=frequency_penalty
      )
      ai_texts.append(response.choices[0].message['content'].strip())
    return ai_texts



In [ ]:
#without gradient
# input will contain the raw output scores(logits in this case) of the probabilities generated by the detector
class textPrediction:
  # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  @staticmethod
  def textInputFun(examples):
    with torch.no_grad():
      inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
      if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        detector.resize_token_embeddings(len(tokenizer))
      input = {k: v.to(device) for k, v in inputs.items()}
      output_prob = F.log_softmax(detector(**inputs).logits, -1)[:, 0].exp().tolist()
    print("No of input instances fed to the detector", len(examples))
    print(f"Probability of text examples: {output_prob}")
    return output_prob

In [ ]:
api_key = "your-openai-key"
model = "gpt-3.5-turbo"
nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=0, temperature=0.5,top_p=0.1, n=1, frequency_penalty=0)
print(AI_texts)

['Trump has expressed his willingness to meet with North Korean leader Kim Jong Un in the near future.', 'such as steam generation, water heating, and thermal energy storage. These technologies use mirrors or lenses to concentrate sunlight onto a receiver, where the energy is converted into heat. This heat can then be used in various industrial processes such as food processing, chemical production, and desalination. By harnessing the power of the sun, these solar concentrating technologies offer a sustainable and environmentally friendly alternative to traditional fossil fuel-based heating systems.', '...action against offshore oil drilling in California. The bill aims to protect the coastal communities and marine environment from the potential risks associated with oil drilling activities.', 'The Bush administration then turned its attention to Iraq, arguing that the need to remove Saddam Hussein from power had become urgent.']


In [ ]:
class ParaphraseNLG:
  @staticmethod
  def openai_response(text, openai_model):
    instruction = {"role": "system", "content": "Enhance the word choices in the sentence to sound more like that of a human."}
    prompt={"role": "user", "content": text}
    messages = [instruction, prompt]
    k_wargs = { "messages": messages, "model": openai_model}
    r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
    return r

In [ ]:
#These are the paraphrased responses generated from GPT 3.5
#Using gpt-4o to check the semantical differences from human text
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]

In [ ]:
print("Human text predictions from the detector")
human_preds = textPrediction.textInputFun(human_texts)
print("AI text predictions from the detector")
ai_preds = textPrediction.textInputFun(AI_texts)
print("Paraphrased text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses)

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import openai
import torch.nn.functional as F

#Instruction and text inputs (Paper Reference -[Learning toWatermark LLM-generated Text via Reinforcement Learning] "Examples of C4 on Llama2-7b dataset", Examples of C4 on OPT-1.3b dataset)
instruction = "You will assist in completing the given text examples:"
raw_texts = ["The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald...",
             "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications...",
             "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take...",
             "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent..."]
human_texts = ["The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald Trump and North Korean leader Kim Jong Un will meet within weeks, White House spokesman Raj Shah said on Monday, as the administration seeks a deal with Pyongyang to rid North Korea of its nuclear weapons programme. The meeting would be the first of its kind between leaders of the countries, and potentially set in motion a long-awaited ”peace dialogue” that could lead to the first meeting of Moon Jae-in, the newly elected leader of South Korea, with the reclusive North. Advertisement For the latest headlines, follow our Google News channel online or via the app. The visit of Kim to South Korea for a summit on April",
               "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications. The first commercial system was the Solar Total Energy Project (STEP) in Shenandoah, Georgia, USA where a field of 114 parabolic dishes provided 50% of the process heating, air conditioning and electrical requirements for a clothing factory. This grid-connected cogeneration system provided 400 kW of electricity plus thermal energy in the form of 401 kW steam and 468 kW chilled water, and had a one-hour peak load thermal storage. Evaporation ponds are shallow pools that concentrate dissolved solids through evaporation. The use of evaporation ponds to obtain salt from sea water is one of the oldest applications of solar energy. Modern uses include concentrating brine solutions used in leach mining and removing dissolved solids from waste",
               "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take a bite out of the cost of living in the South Bay. The bill, AB 191, would allow cities to create a Community Facilities District, which would allow them to issue bonds to pay for infrastructure improvements. The bill would also allow cities to create a Community Facilities District to pay for infrastructure improvements. “The South Bay is a great place to live, work and raise a family, but the cost of living is too high,” Muratsuchi said in a statement. “AB 191 will help cities in the South Bay and across the state",
               "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent. Among the stated reasons were that Saddam's regime had tried to acquire nuclear material and had not properly accounted for biological and chemical material it was known to have previously possessed, and believed to still maintain. Both the possession of these weapons of mass destruction (WMD), and the failure to account for them, would violate the U.N. sanctions. The assertion about WMD was hotly advanced by the Bush administration from the beginning, but other major powers including China, France, Germany, and Russia remained unconvinced that Iraq was a threat and refused to allow passage of a UN Security Council resolution to authorize the use of force. Iraq permitted UN weapon inspectors in November 2002, who were continuing their work to assess the WMD claim when the Bush administration decided to proceed with war without UN authorization and told the inspectors to leave the"]

# Setup
device = "cuda:0"
detector_modelcard = "databricks/dolly-v2-3b"
detector = AutoModelForSequenceClassification.from_pretrained(detector_modelcard)
tokenizer = AutoTokenizer.from_pretrained(detector_modelcard)
# detector.to(device)
detector.eval()

# Define classes
class NLGClass:
    def __init__(self, api_key, raw_texts, instruction, model):
        openai.api_key = api_key
        self.human_texts = raw_texts
        self.instruction = instruction
        self.model = model

    def generate_text_completion(self, max_tokens, seed, temperature, top_p, n, frequency_penalty):
        ai_texts = []
        for text in self.human_texts:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.instruction},
                    {"role": "user", "content": text}
                ],
                max_tokens=max_tokens,
                seed=seed,
                temperature=temperature,
                top_p=top_p,
                n=n,  # iterations
                stop=None,
                frequency_penalty=frequency_penalty
            )
            ai_texts.append(response.choices[0].message['content'].strip())
        return ai_texts

class textPrediction:
    @staticmethod
    def textInputFun(examples):
        with torch.no_grad():
            # inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
            # input = {k: v.to(device) for k, v in inputs.items()}
            # output_prob = F.log_softmax(detector(**inputs).logits, -1)[:, 0].exp().tolist()
            inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = detector(**inputs)
            logits = outputs.logits
            output_prob = F.log_softmax(logits, -1)[:, 0].exp().tolist()
        print("No of input instances fed to the detector", len(examples))
        print(f"Probability of text examples: {output_prob}")
        return output_prob

class ParaphraseNLG:
  @staticmethod
  def openai_response(text, openai_model):
    instruction = {"role": "system", "content": "Enhance the word choices in the sentence to sound more like that of a human."}
    prompt={"role": "user", "content": text}
    messages = [instruction, prompt]
    k_wargs = { "messages": messages, "model": openai_model}
    r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
    return r

# Example usage
api_key = "your-openai-key"

model = "gpt-3.5-turbo"

nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=0, temperature=0.5, top_p=0.1, n=1, frequency_penalty=0)
print(AI_texts)

# These are the paraphrased responses generated from GPT 3.5
# Using gpt-4o to check the semantical differences from human text
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]

print("Human text predictions from the detector")
human_preds = textPrediction.textInputFun(raw_texts)

print("AI text predictions from the detector")
ai_preds = textPrediction.textInputFun(AI_texts)

print("Paraphrased text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses)


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import openai
import torch.nn.functional as F

# Instruction and text inputs
instruction = "You will assist in completing the given text examples:"
raw_texts = [
    "The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald...",
    "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications...",
    "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take...",
    "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent..."
]
human_texts = [
    "The White House has said the first meeting ever between sitting U.S. and North Korean leaders could take place in the coming weeks. Washington: President Donald Trump and North Korean leader Kim Jong Un will meet within weeks, White House spokesman Raj Shah said on Monday, as the administration seeks a deal with Pyongyang to rid North Korea of its nuclear weapons programme. The meeting would be the first of its kind between leaders of the countries, and potentially set in motion a long-awaited ”peace dialogue” that could lead to the first meeting of Moon Jae-in, the newly elected leader of South Korea, with the reclusive North. Advertisement For the latest headlines, follow our Google News channel online or via the app. The visit of Kim to South Korea for a summit on April",
    "Solar concentrating technologies such as parabolic dish, trough and Scheffler reflectors can provide process heat for commercial and industrial applications. The first commercial system was the Solar Total Energy Project (STEP) in Shenandoah, Georgia, USA where a field of 114 parabolic dishes provided 50% of the process heating, air conditioning and electrical requirements for a clothing factory. This grid-connected cogeneration system provided 400 kW of electricity plus thermal energy in the form of 401 kW steam and 468 kW chilled water, and had a one-hour peak load thermal storage. Evaporation ponds are shallow pools that concentrate dissolved solids through evaporation. The use of evaporation ponds to obtain salt from sea water is one of the oldest applications of solar energy. Modern uses include concentrating brine solutions used in leach mining and removing dissolved solids from waste",
    "An undated aerial photo during the oil boom era in Hermosa Beach. South Bay Assemblyman Al Muratsuchi on Friday announced a bill designed to take a bite out of the cost of living in the South Bay. The bill, AB 191, would allow cities to create a Community Facilities District, which would allow them to issue bonds to pay for infrastructure improvements. The bill would also allow cities to create a Community Facilities District to pay for infrastructure improvements. “The South Bay is a great place to live, work and raise a family, but the cost of living is too high,” Muratsuchi said in a statement. “AB 191 will help cities in the South Bay and across the state",
    "The Bush administration then turned its attention to Iraq, and argued the need to remove Saddam Hussein from power in Iraq had become urgent. Among the stated reasons were that Saddam's regime had tried to acquire nuclear material and had not properly accounted for biological and chemical material it was known to have previously possessed, and believed to still maintain. Both the possession of these weapons of mass destruction (WMD), and the failure to account for them, would violate the U.N. sanctions. The assertion about WMD was hotly advanced by the Bush administration from the beginning, but other major powers including China, France, Germany, and Russia remained unconvinced that Iraq was a threat and refused to allow passage of a UN Security Council resolution to authorize the use of force. Iraq permitted UN weapon inspectors in November 2002, who were continuing their work to assess the WMD claim when the Bush administration decided to proceed with war without UN authorization and told the inspectors to leave the"
]

# Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
detector_modelcard = "databricks/dolly-v2-3b"
detector = AutoModelForSequenceClassification.from_pretrained(detector_modelcard)
tokenizer = AutoTokenizer.from_pretrained(detector_modelcard)

# Ensure the padding token is set
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    detector.resize_token_embeddings(len(tokenizer))
    detector.config.pad_token_id = tokenizer.pad_token_id

detector.to(device)
detector.eval()

# Define classes
class NLGClass:
    def __init__(self, api_key, raw_texts, instruction, model):
        openai.api_key = api_key
        self.human_texts = raw_texts
        self.instruction = instruction
        self.model = model

    def generate_text_completion(self, max_tokens, seed, temperature, top_p, n, frequency_penalty):
        ai_texts = []
        for text in self.human_texts:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.instruction},
                    {"role": "user", "content": text}
                ],
                max_tokens=max_tokens,
                seed=seed,
                temperature=temperature,
                top_p=top_p,
                n=n,  # iterations
                stop=None,
                frequency_penalty=frequency_penalty
            )
            ai_texts.append(response.choices[0].message['content'].strip())
        return ai_texts

class textPrediction:
    @staticmethod
    def textInputFun(examples):
        with torch.no_grad():
            inputs = tokenizer(examples, padding=True, truncation=True, max_length=300, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = detector(**inputs)
            logits = outputs.logits
            output_prob = F.log_softmax(logits, -1)[:, 0].exp().tolist()
        print("No of input instances fed to the detector", len(examples))
        print(f"Probability of text examples: {output_prob}")
        return output_prob

class ParaphraseNLG:
    @staticmethod
    def openai_response(text, openai_model):
        instruction = {"role": "system", "content": "Enhance the word choices in the sentence to sound more like that of a human."}
        prompt = {"role": "user", "content": text}
        messages = [instruction, prompt]
        k_wargs = {"messages": messages, "model": openai_model}
        r = openai.ChatCompletion.create(**k_wargs)['choices'][0].message.content
        return r

# Example usage
api_key = "your-openai-key"
model = "gpt-3.5-turbo"

nlg_obj = NLGClass(api_key, raw_texts, instruction, model)
AI_texts = nlg_obj.generate_text_completion(max_tokens=300, seed=0, temperature=0.5, top_p=0.1, n=1, frequency_penalty=0)
print(AI_texts)

# These are the paraphrased responses generated from GPT 3.5
# Using gpt-4o to check the semantical differences from human text
Paraphrased_responses = [ParaphraseNLG.openai_response(i, "gpt-4o") for i in AI_texts]

print("Human text predictions from the detector")
human_preds = textPrediction.textInputFun(human_texts)

print("AI text predictions from the detector")
ai_preds = textPrediction.textInputFun(AI_texts)

print("Paraphrased text predictions")
paraphrased_preds = textPrediction.textInputFun(Paraphrased_responses)
